In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
from pprint import pprint
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
files.upload()

In [15]:
import pandas as pd
df = pd.read_csv('JC-202102-citibike-tripdata.csv') 
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,FD73FB85F008349D,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member
2,39F9E6663CB5FDF6,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194,member
3,A64745CB0792EC6F,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,casual
4,75CC76EB9543764A,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,member


In [16]:
df.describe()


,start_lat,start_lng,end_lat,end_lng
count,4881.000000,4881.000000,4813.000000,4813.000000
mean,40.722785,-74.047173,40.722589,-74.046511
std,0.006944,0.011901,0.006793,0.011672
min,40.711130,-74.083639,40.703554,-74.083639
25%,40.717732,-74.051788,40.717732,-74.050990
50%,40.721650,-74.044247,40.721525,-74.043845
75%,40.727551,-74.038050,40.727224,-74.037683
max,40.748716,-74.032108,40.754530,-73.977706


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4881 entries, 0 to 4880
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ride_id             4881 non-null   object 
 1   rideable_type       4881 non-null   object 
 2   started_at          4881 non-null   object 
 3   ended_at            4881 non-null   object 
 4   start_station_name  4881 non-null   object 
 5   start_station_id    4881 non-null   object 
 6   end_station_name    4813 non-null   object 
 7   end_station_id      4813 non-null   object 
 8   start_lat           4881 non-null   float64
 9   start_lng           4881 non-null   float64
 10  end_lat             4813 non-null   float64
 11  end_lng             4813 non-null   float64
 12  member_casual       4881 non-null   object 
dtypes: float64(4), object(9)
memory usage: 495.9+ KB


In [18]:
df.isnull().sum()

ride_id                0
rideable_type          0
started_at             0
ended_at               0
start_station_name     0
start_station_id       0
end_station_name      68
end_station_id        68
start_lat              0
start_lng              0
end_lat               68
end_lng               68
member_casual          0
dtype: int64

In [22]:
df[df['started_at'].isnull()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual


In [23]:
df[df['ended_at'].isnull()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual


In [24]:
df = df[:-1]
df.isnull().sum()


ride_id                0
rideable_type          0
started_at             0
ended_at               0
start_station_name     0
start_station_id       0
end_station_name      68
end_station_id        68
start_lat              0
start_lng              0
end_lat               68
end_lng               68
member_casual          0
dtype: int64

In [31]:
print(type(df["start_lat"][0]))
print(df["start_lat"][0])

<class 'numpy.float64'>
40.735208


In [32]:
df['start_lat'].unique()

array([40.735208, 40.728744, 40.73367 , 40.721525, 40.734961, 40.724294,
       40.71942 , 40.71649 , 40.714584, 40.712418, 40.714358, 40.725685,
       40.723658, 40.711242, 40.742677, 40.748715, 40.719282, 40.730897,
       40.725339, 40.71113 , 40.719251, 40.714145, 40.721123, 40.718211,
       40.727551, 40.722104, 40.737603, 40.731168, 40.723332, 40.723331,
       40.722103, 40.728745, 40.712419, 40.731169, 40.72534 , 40.71654 ,
       40.727224, 40.727223, 40.713464, 40.724176, 40.721124, 40.713463,
       40.726012, 40.727595, 40.716539, 40.737711, 40.726011, 40.727596,
       40.744318, 40.737604, 40.725726, 40.748716, 40.719586, 40.715177,
       40.734785, 40.71687 , 40.718489, 40.718488, 40.719252, 40.715178,
       40.723659, 40.724177, 40.745909, 40.744319, 40.712774, 40.724605,
       40.734786, 40.721651, 40.72165 , 40.72163 , 40.717732, 40.717733,
       40.725289, 40.718355])

In [29]:
def camel_case(city):
    try:
        city = city.split(' ')
        city = ' '.join([x.lower().capitalize() for x in city])
        if city == 'Unknown':
            return np.nan
        else:
            return city
    except:
        return np.nan
    
# Apply camel_case function to City column
df['start_station_name'] = df['start_station_name'].apply(camel_case)
df['start_station_name'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Newport Pkwy                  298
Grove St Path                 283
Hamilton Park                 271
Marin Light Rail              233
Sip Ave                       229
Newport Path                  206
Warren St                     167
Harborside                    164
Jc Medical Center             160
Columbus Dr At Exchange Pl    157
City Hall                     139
Mcginley Square               135
Liberty Light Rail            130
Newark Ave                    119
Washington St                 110
Grand St                      108
Columbus Drive                 99
Montgomery St                  93
Brunswick St                   92
Morris Canal                   89
Van Vorst Park                 88
Manila & 1st                   87
Jersey & 3rd                   85
Brunswick & 6th                85
Paulus Hook                    82
Baldwin At Montgomery          79
Jersey & 6th St                74
Monmouth And 6th               74
Fairmount Ave                  70
Journal Square

In [30]:
df.count()

ride_id               4880
rideable_type         4880
started_at            4880
ended_at              4880
start_station_name    4880
start_station_id      4880
end_station_name      4812
end_station_id        4812
start_lat             4880
start_lng             4880
end_lat               4812
end_lng               4812
member_casual         4880
dtype: int64

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4880 entries, 0 to 4879
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ride_id             4880 non-null   object 
 1   rideable_type       4880 non-null   object 
 2   started_at          4880 non-null   object 
 3   ended_at            4880 non-null   object 
 4   start_station_name  4880 non-null   object 
 5   start_station_id    4880 non-null   object 
 6   end_station_name    4812 non-null   object 
 7   end_station_id      4812 non-null   object 
 8   start_lat           4880 non-null   float64
 9   start_lng           4880 non-null   float64
 10  end_lat             4812 non-null   float64
 11  end_lng             4812 non-null   float64
 12  member_casual       4880 non-null   object 
dtypes: float64(4), object(9)
memory usage: 495.8+ KB


In [40]:
df.shape

(4880, 13)

In [43]:
df.to_csv('New_dataset.csv', index=False)